In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import gdown
import ee
import google
import os
import geemap

from tobler.util import h3fy
from ee_jupyter.ipyleaflet import Map

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1Adeu5BXT4R5NZOOb6lEQPRdFqffFZ0sg3dZ6vtUp3rSbhLO6rZQBJX5Ms3w

Successfully saved authorization token.


In [7]:
list1 = ee.List.sequence(0, 364, 1);
list2 = ee.List.sequence(0,23,1);

def outer(e1, acc):
    pairs = list2.map(lambda e2: [e1, e2])
    return ee.List(acc).cat(pairs)

combinations = ee.List(
  list1.iterate(outer, ee.List([])))
combinations.size().getInfo(), combinations.get(1).getInfo()

(8760, [0, 1])

In [9]:
# // Define a function to apply on each combination
def getMeanHourlyPerDay(pair):
#     // Cast pairList to an Earth Engine List
    pairList = ee.List(pair);
#     // Extract e1 and e2
    day = pairList.get(0); 
    hour = pairList.get(1); 
    
#     // Your desired operation using e1 and e2
#     // Here's a simple example that just multiplies e1 and e2
    return tempClip.filter(ee.Filter.dayOfYear(day, day)).filter(ee.Filter.eq('hour', hour)).mean();

In [10]:
# EXTRACT Juneau and temperature
juneau = ee.FeatureCollection("TIGER/2018/Counties").filter(ee.Filter.eq('COUNTYFP', '110')).filter(ee.Filter.eq('STATEFP', '02'))
juneauGeometry = juneau.geometry()
juneauBounds = juneauGeometry.bounds()

temp = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY").filterDate('2010-01-01', '2019-12-31').select('temperature_2m')

tempClip = temp.map(lambda image: image.clip(juneauBounds))

# // Map your function over the combinations
results = combinations.map(getMeanHourlyPerDay);

resultsCollection = ee.ImageCollection.fromImages(results)
# print(resultsCollection);

In [11]:
task = ee.batch.Export.table.toDrive(**{
  'collection': resultsCollection,
  'description': 'juneau temp 10 year averages not blocks from erin',
  'folder': 'EarthEngine',
  #'selectors': ['BLOCK','FIPS', 'date', 'mean', 'temperature_2m'], 
  'fileFormat': 'CSV'
})
task.start()

In [20]:
os.path.exists('../DSSG2023-Heating-Loads-Data/shapefiles/Blocks2020/Blocks2020.shp')

True

In [21]:
blocks = gpd.read_file('../DSSG2023-Heating-Loads-Data/shapefiles/Blocks2020/Blocks2020.shp')
blocks = blocks[['NAME', 'STATE', 'BOROUGH', 'TRACT', 'BLOCK', 'FIPS', 'TOTALPOP', 'HOUSEUNITS', 'geometry']]
juneauBlocks = blocks[(blocks['BOROUGH'] == '110') & (blocks['TOTALPOP'] > 0)]
juneauBlocksFC = geemap.geopandas_to_ee(juneauBlocks)

In [22]:
def reduce_to_region(image):
    reduced_image = image.reduceRegions(collection=juneauBlocksFC, reducer=ee.Reducer.mean(), scale=1000, crs = 'EPSG:4326')
    #reduced_image = reduced_image.set('date', image.get('julian_date'))
    return reduced_image

export = resultsCollection.map(reduce_to_region)

In [23]:
task = ee.batch.Export.table.toDrive(**{
  'collection': export,
  'description': 'juneau temp 10 year averages with blocks from erin',
  'folder': 'EarthEngine',
  #'selectors': ['BLOCK','FIPS', 'date', 'mean', 'temperature_2m'], 
  'fileFormat': 'CSV'
})
task.start()

In [249]:
CGAZ_ADM1 = ee.FeatureCollection('projects/earthengine-legacy/assets/projects/sat-io/open-datasets/geoboundaries/CGAZ_ADM1')

# Filter Comprehensive Global Administrative Zones ADM1 for AK
ak = CGAZ_ADM1.filter(ee.Filter.eq('shapeISO', 'US-AK'))
# Convert the feature collection to a geometry.
akGeometry = ak.geometry()
# Get the bounding box of the geometry.
akBounds = akGeometry.bounds()

# same process for Juneau
juneau = ee.FeatureCollection("TIGER/2018/Counties").filter(ee.Filter.eq('COUNTYFP', '110')).filter(ee.Filter.eq('STATEFP', '02'))
juneauGeometry = juneau.geometry()
juneauBounds = juneauGeometry.bounds()

In [250]:
temp = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY").filterDate('2010-01-01', '2019-12-31').select('temperature_2m').map(lambda image: image.clip(juneauGeometry))

In [334]:
#temp.first().getInfo()

In [394]:
def temporalReduction(temp):
    # create a list for every day-hour combo
    dates = []
    for d in range(2, 3):
        for h in range(0, 24):
            dates.append((d, h))
    
    # empty list to store the 10 year average images
    collection = []
    # calculate averages for every day-hour combo
    for day, hour in dates:
        average = temp.filter(ee.Filter.dayOfYear(day, day)).filter(ee.Filter.eq('hour', hour)).mean()
        collection.append(average)
        
    return ee.ImageCollection(collection)

temp_10year_average = temporalReduction(temp)

temp_10year_average = temp_10year_average.filter(ee.Filter.eq('temperature_2m', []))

type(temp_10year_average), type(temp_10year_average.first())

(ee.imagecollection.ImageCollection, ee.image.Image)

In [395]:
task = ee.batch.Export.table.toDrive(**{
  'collection': temp_10year_average,
  'description': 'juneau temp 10 year averages filtered by area 4',
  'folder': 'EarthEngine',
  #'selectors': ['BLOCK','FIPS', 'date', 'mean', 'temperature_2m'], 
  'fileFormat': 'CSV'
})
task.start()

In [325]:
one_hour = temp.filter(ee.Filter.dayOfYear(2, 2)).filter(ee.Filter.eq('hour', 2)).mean()

In [382]:
type(one_hour)

ee.image.Image

In [376]:
map1 = Map(center=[58.3005, -134.4201], zoom=10)

map1.addLayer(one_hour, {'min': 0, 'max': 272})

map1

Map(center=[58.3005, -134.4201], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', '…

In [362]:
buildings = ee.FeatureCollection('projects/sat-io/open-datasets/ORNL/USA-STRUCTURES/USA_ST_AK')

buildingsRaster = ee.Image().byte().paint(featureCollection = buildings, color = 1)

# Get coverage of buildings per ERA-5 Pixel
imageBuildings = buildingsRaster.unmask(0).setDefaultProjection('EPSG:4326', None, 100).reproject('EPSG:4326', None, 100).reduceResolution(
      reducer = ee.Reducer.mean(), maxPixels = 12393).multiply(ee.Image.pixelArea()).reproject('EPSG:4326', None, 11132)


# map1 = Map(center=[58.3005, 134.4201], zoom=10)

# map1.addLayer(imageBuildings, {}) 
# map1

In [280]:
type(imageBuildings)

ee.image.Image

In [257]:
centroids = imageBuildings.gt(0).sample(
  region = akGeometry,
  geometries =  True);

Notes from convo with Erin:

join centroids (small blocks) or imageBuildings mask (larger blocks)

updateMask() function mapped on 10year average, then unmask 0, then spatially reduce

In [327]:
def maskTemp(image):
    return image.updateMask(imageBuildings.reproject('EPSG:4326', None, 11132)).unmask(0)

temp_10year_average_masked = temp_10year_average.map(maskTemp)

In [328]:
type(temp_10year_average_masked), type(temp_10year_average_masked.first())

(ee.imagecollection.ImageCollection, ee.image.Image)

In [341]:
task = ee.batch.Export.table.toDrive(**{
  'collection': temp_10year_average_masked,
  'description': 'juneau temp 10 year averages masked',
  'folder': 'EarthEngine',
  #'selectors': ['BLOCK','FIPS', 'date', 'mean', 'temperature_2m'], 
  'fileFormat': 'CSV'
})
task.start()

In [263]:
blocks = gpd.read_file('../DSSG2023-Heating-Loads-Data/shapefiles/Blocks2020/Blocks2020.shp')
blocks = blocks[['NAME', 'STATE', 'BOROUGH', 'TRACT', 'BLOCK', 'FIPS', 'TOTALPOP', 'HOUSEUNITS', 'geometry']]
juneauBlocks = blocks[(blocks['BOROUGH'] == '110') & (blocks['TOTALPOP'] > 0)]
juneauBlocksFC = geemap.geopandas_to_ee(juneauBlocks)

In [264]:
juneauBlocksFC.size().getInfo()

380

In [265]:
areas = juneauBlocks['geometry'].area
areas.min(), areas.median(), areas.max()

(1055.8870653038284, 27503.41284678245, 2220401169.367476)

In [370]:
# spatial reduction according to juneauBlocksFC

def reduce_to_region(image):
    reduced_image = image.reduceRegions(collection=juneauBlocksFC, reducer=ee.Reducer.mean(), scale=1000, crs = 'EPSG:4326')
    #reduced_image = reduced_image.set('date', image.get('julian_date'))
    return reduced_image

export = temp_10year_average.map(reduce_to_region)

In [365]:
type(temp_10year_average)

ee.imagecollection.ImageCollection

In [383]:
export_one_hour = one_hour.reduceRegions(collection=juneauBlocksFC, reducer=ee.Reducer.mean(), scale=1000, crs = 'EPSG:4326')

In [350]:
type(export), type(export.first())

(ee.imagecollection.ImageCollection, ee.image.Image)

In [371]:
export1 = export.first()

In [372]:
task = ee.batch.Export.table.toDrive(**{
  'collection': export1,
  'description': 'juneau block export for first block no mask',
  'folder': 'EarthEngine',
  #'selectors': ['BLOCK','FIPS', 'date', 'mean', 'temperature_2m'], 
  'fileFormat': 'CSV'
})
task.start()

In [373]:
task = ee.batch.Export.table.toDrive(**{
  'collection': export,
  'description': 'juneau block export try 1 no mask',
  'folder': 'EarthEngine',
  #'selectors': ['BLOCK','FIPS', 'date', 'mean', 'temperature_2m'], 
  'fileFormat': 'CSV'
})
task.start()